In [2]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)

## give the continuous columns:
## Please note that this is a inbuilt dataset in tensorFlow model
## credits: https://www.tensorflow.org/tutorials/wide_and_deep


In [ ]:
train_df = pd.read_csv("train_file.csv")

continous_features = [];
for each_column in train_df.columns:
    if train_df[each_column].dtype == 'float':
        each_column = tf.feature_column.numeric_column(each_column)
        continous_features.append(each_column)

## Give the categorical columns and the base columns

In [ ]:
cat_features = []; base_columns = []
for each_column in train_df.columns:
    if train_df[each_column].dtype == 'str' or train_df[each_column].dtype == 'category':
        base_columns.append(each_column)
        each_column = tf.feature_column.embedding_column(each_column, dimension=8) #dimension is tunable
        cat_features.append(each_column)
        
        


## Crossed Columns: On combining features into a single feature, the model will be able to earn seperate weights for the combinations

In [ ]:
def cross_column_creator(base_columns):
    
    cross_columns
    for i in range (len(base_columns)):
        cross_columns.append(tf.feature_column.crossed_column(
        [base_columns[i], base_columns[i+1]], hash_bucket_size=1000))


## Combining all the columns to create the deep columns list

In [ ]:
deep_column = []
for each_column in cat_features:
    deep_column.append(each_column)
for each_column in continous_features:
    deep_column.append(each_column)

## Creating the model:

In [ ]:
model = tf.estimator.DNNLinearCombinedRegressor(
    model_dir='path',
    linear_feature_columns=base_columns + crossed_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[1, 2])#tunable units

## Training and evaluating the model

In [ ]:
for n in range(iterations):
  model.train(dnn_optimizer=tf.train.ProximalAdagradOptimizer(...),tunable_params)
results = model.evaluate(input_fn=lambda: input_fn(
      FLAGS.test_data, 1, False, FLAGS.batch_size))


## Normal deep Neural Networks

## Creating place holders for the input data

In [6]:
def create_placeholders(n_x, n_y):
    
    X = tf.placeholder(tf.float32,name = 'X')
    Y = tf.placeholder(tf.float32,name = 'Y')
   
    return X, Y

## Random initialization of Parameters for every layer

In [7]:

def initialize_parameters(layer_dims):
    """
    Initializes parameters to build a neural network with tensorflow. 
            
    Returns:
    parameters -- a dictionary of tensors.
    """
    
    tf.set_random_seed(1)                  
        
    parameters = {}
    
    for l in range(len(layer_dims)):
        parameters["W"+str(l+1)] = tf.get_variable("W", [l,l+1], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        parameters["b"+str(l+1)] = tf.get_variable("b", [l,1], initializer = tf.zeros_initializer())

    
    return parameters

## Implementing the forward propogation module

In [ ]:
def forward_propagation(X, parameters,number_of_layers,Y):   
    
    temp = X; Z= None;
    for l in range(number_of_layers):
        Z = tf.add(tf.matmul(parameters['W'+str((l+1))],temp),'b'+str((l+1)))
        A = tf.nn.relu('Z'+str(l+1))
        temp = A
    
    cost = tf.reduce_mean(tf.square(Z - Y))
    
    return Z, cost

## Model Training

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,layer_dims,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    
    
    ops.reset_default_graph()                         
                                      
    (n_x, m) = X_train.shape                          
    n_y = Y_train.shape[0]                            
    costs = []                                       
    X, Y = create_placeholders(n_x, n_y)
    parameters = initialize_parameters (layer_dims)
    Z = forward_propagation(X, parameters)[0]
    cost = forward_propagation(X, parameters)[1]
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):

            epoch_cost = 0.                       
            num_minibatches = int(m / minibatch_size) 
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

    return parameters